# Prepare Data

In [1]:
import pandas as pd , numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler,OneHotEncoder,RobustScaler
import streamlit as st

In [2]:
df = pd.read_excel ("customers.xlsx")

In [3]:
df.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CLIENTNUM        10127 non-null  int64 
 1   Attrition_Flag   10127 non-null  object
 2   Customer_Age     10127 non-null  int64 
 3   Gender           10127 non-null  object
 4   Dependent_count  10127 non-null  int64 
 5   Education_Level  10127 non-null  object
 6   Marital_Status   10127 non-null  object
 7   Income           10127 non-null  int64 
 8   Card_Category    10127 non-null  object
 9   Months_on_book   10127 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 791.3+ KB


In [4]:
n_cols = df.select_dtypes(include=["int64","float64"]).columns.tolist()
c_cols = df.select_dtypes(exclude=["int64","float64"]).columns.tolist()
cl = {1: "High-Value Customers",2: "Medium-Value Customers",0: "Low-Value Customers"}

In [5]:
encoder = OneHotEncoder (drop="first",handle_unknown="ignore",sparse_output=False)
df_encoder = encoder.fit_transform (df[c_cols])
df_encoder = pd.DataFrame (df_encoder,columns=encoder.get_feature_names_out(c_cols))
df_encoder

,Attrition_Flag_Existing Customer,Gender_Male,Education_Level_Doctorate,Education_Level_Graduate,Education_Level_High School,Education_Level_Post-Graduate,Education_Level_Uneducated,Education_Level_Unknown,Marital_Status_Married,Marital_Status_Single,Marital_Status_Unknown,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10123,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10124,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10125,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
scaler = StandardScaler ()
df_scaler = scaler.fit_transform (df[n_cols])
df_scaler = pd.DataFrame(df_scaler,columns=n_cols)
df_scaler.head()

,CLIENTNUM,Customer_Age,Dependent_count,Income,Months_on_book
0,0.802878,-0.165406,0.503368,0.496508,0.384621
1,2.156861,0.333570,2.043199,-0.661515,1.010715
2,-0.682769,0.583058,0.503368,1.365026,0.008965
3,0.832862,-0.789126,1.273283,-0.661515,-0.241473
4,-0.814896,-0.789126,0.503368,0.496508,-1.869317


In [7]:
df_final = pd.concat ((df_scaler,df_encoder),axis=1)

# Model

In [8]:
model = KMeans (n_clusters=3,random_state=42)

In [9]:
model.fit (df_final)

KMeans(n_clusters=3, random_state=42)

In [10]:
df["Cluster"] = model.labels_
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income,Card_Category,Months_on_book,Cluster
0,768805383,Existing Customer,45,Male,3,High School,Married,70000,Blue,39,1
1,818770008,Existing Customer,49,Female,5,Graduate,Single,30000,Blue,44,1
2,713982108,Existing Customer,51,Male,3,Graduate,Married,100000,Blue,36,1
3,769911858,Existing Customer,40,Female,4,High School,Unknown,30000,Blue,34,1
4,709106358,Existing Customer,40,Male,3,Uneducated,Married,70000,Blue,21,0
